In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))

In [ ]:
import json 
from pprint import pprint 
import pandas as pd
import numpy as np
from pathlib import Path
import os
import re

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} JSON files in '{dirpath}'.")

In [ ]:
data_path = Path("ALGORITM/")
walk_through_dir(data_path)

In [ ]:
# Get all file paths - все файлы в папке ALGORITM
file_path_list = list(data_path.glob("*.json"))

i = 0
file_id=list()
print(type(file_id))
for file in file_path_list: 
   
    print(f"{i+1} file path: {file_path_list[i]}")
    fp = str(file_path_list[i])
    file_id.append(fp[-10:-5])
    i+=1

In [ ]:
k = 0
load_data = pd.DataFrame()
ld = pd.DataFrame()
for file in file_path_list: 
    file_path = 'ALGORITM/data_for_algoritm_' + file_id[k] + '.json'
    load_file = 'da_'+ file_id[k]
    with open(file_path, encoding='utf-8') as f:
        load_file = json.load(f) 
    df2 = pd.json_normalize(load_file, max_level=3).melt()
    
 # вывод таблиц с размерами и свойствами коробок, а их дофига...
    df4 = pd.json_normalize(df2.loc[df2["variable"] == "data_result.boxes","value"].iloc[0], max_level=0).melt()
    cargo = pd.DataFrame()
    cargo['size'] = df4.loc[df4['variable'] == 'size', ['value']].reset_index(drop=True)
    stacking = df4.loc[df4['variable'] == 'stacking', ['value']].reset_index(drop=True)
    turnover = df4.loc[df4['variable'] == 'turnover', ['value']].reset_index(drop=True)
    cargo['turnover'] = turnover['value']
    cargo['stacking'] = stacking['value'] 
    #display(cargo)  # для вывода всех таблиц с размерами и свойствами коробок разкомментируйте эту строку

 # создание таблицы свойств карго со всех файлов  
    load_data = [{'load_id':file_id[k],'density_percent':df2.loc[df2["variable"] == "data_result.cargo_space.calculation_info.density_percent","value"].iloc[0], 
                 'filling_space_percent':df2.loc[df2["variable"] == "data_result.cargo_space.calculation_info.filling_space_percent","value"].iloc[0], 
                 'loading_width':df2.loc[df2["variable"] == "data_result.cargo_space.loading_size.width","value"].iloc[0],
                 'loading_height':df2.loc[df2["variable"] == "data_result.cargo_space.loading_size.height","value"].iloc[0], 
                 'loading_length':df2.loc[df2["variable"] == "data_result.cargo_space.loading_size.length","value"].iloc[0], 'boxes_qty':len(cargo)}]
              
    ld = pd.concat((ld, pd.DataFrame(load_data)))
    k+=1

In [ ]:
 # вывод таблицы свойств карго со всех файлов 
ld.reset_index(drop=True)